In [ ]:
# Data visualization tools.
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("whitegrid")

# Has multi-dimensional arrays and matrices. Has a large collection of
# mathematical functions to operate on these arrays.
import numpy as np

from scipy.stats import gaussian_kde

# Data manipulation and analysis.
import pandas as pd

# for survival analysis
from lifelines import CoxPHFitter, KaplanMeierFitter
from lifelines.plotting import add_at_risk_counts

# to be able to clear output
from IPython.display import clear_output

# Enables autoreloading of specified modules.
%load_ext autoreload 
%autoreload 1

# import the allocation model
from kidney_dtsim.model import *
from kidney_dtsim.helper_functions import *

# specify model to be autoreloaded
%aimport kidney_dtsim

# if data 
try:
    data_path # type: ignore
except NameError:
    data_path = "../model_input_public/"

# Loading the data

We load the data for the patients listed 2006-01-01. In the next step we will use this data, to add the initial patients to the model.

In [ ]:
df_active_waiting_2006 = pd.read_csv(data_path + "tidy_waiting_list_example.csv", sep = ";", decimal=",", index_col = 0)

## Predicting waiting list removal probability
First we load the data for the age at removal from the waiting list.
Removal from waiting list is considered an event. Patients were censored, if transplanted.

In [ ]:
df_removal_surv = pd.read_csv(data_path + "removal_survival.csv",
                              sep=";", index_col=0, decimal=",")
df_removal_surv.dropna(inplace=True)
df_removal_surv[df_removal_surv["event_time"] < 0]
df_removal_surv.drop(labels= df_removal_surv[df_removal_surv["event_time"] < 0].index, axis="index", inplace=True)
df_removal_surv.loc[df_removal_surv["event_time"] <= 0, "event_time"] += 1

df_removal_surv["no_zero_time_dial_to_registration"] = df_removal_surv["time_dial_to_registration"].apply(lambda x: 1 if x <= 0 else x)
df_removal_surv["sqrt_time_dial_to_registration"] = np.sqrt(df_removal_surv["no_zero_time_dial_to_registration"])

def assign_age_group(age):
    if age <= 29:
        return "0-29"
    elif age <= 39:
        return "30-39"
    elif age <= 44:
        return "40-44"
    elif age <= 49:
        return "45-49"
    elif age <= 54:
        return "50-54"
    elif age <= 59:
        return "55-59"
    elif age <= 64:
        return "60-64"
    elif age <= 69:
        return "65-69"
    elif age <= 74:
        return "70-74"
    elif age <= 79:
        return "75-79"
    else:
        return "80+"

df_removal_surv["age_group"] = df_removal_surv["age_at_reg_waiting_list"].apply(assign_age_group)

### Fit a Cox Proportional Hazards Model with parametric baseline

A Cox Proportional Hazards model was selected, with a parametric baseline fitted with cubic splines.

In [ ]:
cph_spline = CoxPHFitter(baseline_estimation_method="spline", n_baseline_knots=4)
cph_spline.fit(df_removal_surv, duration_col="event_time", event_col="event", entry_col="entry_time", strata = "age_group", formula="no_zero_time_dial_to_registration + sqrt_time_dial_to_registration")

cph_spline.print_summary(style='ascii') # if not style ascii the formating is lost in the rendered document.

We compare the model to the Kapplan-Meier Estimate. Shown is the 'survival' probability, which means the probability of not being removed from the waiting list.

In [ ]:
def plot_model_to_kmf(model, df):
    # Plot Kaplane-Meier Fitter
    kmf = KaplanMeierFitter()
    kmf.fit(durations=df["event_time"], event_observed=df["event"], entry=df["entry_time"])
    ax = plt.subplot()
    kmf.plot_survival_function(ax=ax)
    plt.tight_layout()

    # Plot other Survival Model
    model.predict_survival_function(df_removal_surv).mean(axis=1).plot(ax=ax)
    
    ax.set_xlabel("days on waiting list")
    ax.set_ylabel("Waiting list 'persistence' probability")
    ax.legend([ax.get_lines()[0], ax.get_lines()[1]], ["Kaplan-Meier Estimate", type(model).__name__ ])
    ax.set_xlim(0, 11000)
    ax.set_yticks(np.arange(0, 1.1, 0.2))
    add_at_risk_counts(kmf, ax = ax)

    plt.show()

plot_model_to_kmf(cph_spline, df_removal_surv)

## Non transplantable and transplantable status probability

In [ ]:
df_nt_status_prob = pd.read_csv(data_path + "nt_status_prob_aalen_johansen.csv", sep = ";", index_col = 0)

df_nt_status_prob.index = pd.to_numeric(df_nt_status_prob.index, errors='coerce')
df_nt_status_prob['pstate.not_transplantable'] = pd.to_numeric(df_nt_status_prob['pstate.not_transplantable'].str.replace(',', '.'), errors='coerce')

print(df_nt_status_prob)

## Load Odds Ratios for organ acceptance
We load odds ratios for organ acceptance of patients in the ETKAS program. The odds ratios are derived from a dataset by Eurotransplant. The logistic regression model is a piecewise logistic regression model.

For the odds-ratio calculation only kidney placed through the regular ETKAS program were evaluated. Candidates whose profile indicated, they didn't want to be offered that graft, were filtered out. The calculation was performed on data from 2016 - 2020 for the ETKAS program and on data from 2014-2019 for the ESP program.

In [ ]:
df_log_odds_ETKAS = pd.read_csv(data_path + "odds_ratios_rd_ETKAS_filtered.csv")
df_log_odds_ETKAS["level"] = df_log_odds_ETKAS["level"].apply(lambda x: str(int(x)) if not pd.isna(x) else "")
df_log_odds_ETKAS["var_name"] = df_log_odds_ETKAS["variable"].fillna('').astype(str) + df_log_odds_ETKAS["variable_transformation"].fillna('').astype(str) + df_log_odds_ETKAS["level"]

dict_log_odds_ETKAS = df_log_odds_ETKAS.set_index("var_name")["coef"].to_dict()
print(dict_log_odds_ETKAS)

In [ ]:
df_log_odds_ESP = pd.read_csv(data_path + "odds_ratios_rd_ESP_filtered_modern.csv")
df_log_odds_ESP["level"] = df_log_odds_ESP["level"].apply(lambda x: str(int(x)) if not pd.isna(x) else "")
df_log_odds_ESP["var_name"] = df_log_odds_ESP["variable"].fillna('').astype(str) + df_log_odds_ESP["variable_transformation"].fillna('').astype(str) + df_log_odds_ESP["level"]

dict_log_odds_ESP = df_log_odds_ESP.set_index("var_name")["coef"].to_dict()
print(dict_log_odds_ESP)

## Load HLA frequencies

The HLA frequencies are derived from http://www.allelefrequencies.net/pop6001c.asp?pop_id=3089 . The selected population was "Germany pop 8".
- Sample size: 39689 (A = 39654, B = 39496, DRB1 = 33683)

In [ ]:
hla_a_freq = pd.read_csv(data_path + "hla_a_freq.csv")
hla_a_freq = hla_a_freq.set_index("match_broad")["freq"].to_dict()
print(hla_a_freq)

hla_b_freq = pd.read_csv(data_path + "hla_b_freq.csv")
hla_b_freq = hla_b_freq.set_index("match_broad")["freq"].to_dict()
print(hla_b_freq)

hla_dr_freq = pd.read_csv(data_path + "hla_drb1_freq.csv")
hla_dr_freq = hla_dr_freq.set_index("match_split")["freq"].to_dict()
print(hla_dr_freq)

## Load HLA types

The HLA hla types are derived from the supplementary files of the following article: Seitz S, Lange V, Norman PJ, Sauter J, Schmidt AH. Estimating HLA haplotype frequencies from homozygous individuals - A Technical Report. Int J Immunogenet. 2021 Dec;48(6):490-495. doi: 10.1111/iji.12553. Epub 2021 Sep 27. PMID: 34570965
- Sample size: 3456066

In [ ]:
df_hla_haplo_freq = pd.read_csv(data_path + "hla_haplo_freq_split.csv", index_col=0)
print(df_hla_haplo_freq)

On the waiting list are cases, where the hla type is only partially reported or not reported at all. The missing hla types are imputed based on the haplotype frequencies in germany. If the hla type is only partially reported or unacceptable antigens are reported, the haplotype frequencies are first filtered for the reported HLAs and for haplotypes not present in the unacceptable antigens for that patient.

In [ ]:
def gen_hla_haplotype(df,
                      a_split=None,
                      a_broad=None,
                      b_split=None,
                      b_broad=None,
                      c_split=None,
                      c_broad=None,
                      drb1_split=None,
                      drb1_broad=None,
                      unacceptable_antigens=None,
                      random_numb_generator = np.random):
    """
    Generates a random HLA haplotype based on a given probability distribution.

    Args:
    - df (DataFrame): A dataframe representing the probability distribution of HLA haplotypes.
    - a (str): The HLA antigen to be filtered.
    - b (str): The HLA antigen to be filtered.
    - c (str): The HLA antigen to be filtered.
    - drb1 (str): The HLA antigen to be filtered.
    - random_numb_generator (module): The random number generator to use, defaulting to `np.random`.

    Returns:
    - list: The randomly generated HLA haplotype.

    Notes:
    - If filtering criteria result in no matching haplotypes, the function falls back to the original unfiltered DataFrame.
    - This is first assessed for unacceptable antigens, and then for HLAs.
    - Frequency normalization occurs after filtering to maintain a valid probability distribution for selection.
    """

    df_orig = df.copy(deep=True)
    
    # Filter out rows where any column contains an antigen from unacceptable_antigens
    if not pd.isnull(unacceptable_antigens):
        unacceptable_antigens = set(unacceptable_antigens.split())  # Convert to set for faster lookups
        df_unacc_antigens_filtered = df[~df[["hla_a_split", "hla_a_broad", "hla_b_split", "hla_b_broad", "hla_c_split", "hla_c_broad", "hla_drb1_broad", "hla_drb1_split"]].isin(unacceptable_antigens).any(axis=1)].copy()
    else:
         df_unacc_antigens_filtered = df.copy()

    if len(df_unacc_antigens_filtered["normalized_freq"]) == 0:
            print("Unnacceptable antigens filtering: The following combination is not represented in the HLA haplotype frequencies DataFrame:")
            print(f"a split:{a_split}, a broad:{a_broad}, b split:{b_split}, b broad:{b_broad}, c split:{c_split}, c broad:{c_broad}, drb1:{drb1_split}, drb1:{drb1_broad} unnacceptable antigens:{unacceptable_antigens}")

            # some combinations are not in the haplotype frequency data
            # in these cases a completly new haplotype is generated
            # for this the unfiltered dataframe is used
            df_unacc_antigens_filtered = df_orig.copy()

    filters = [("hla_a_split", a_split), 
           ("hla_a_broad", a_broad),
           ("hla_b_split", b_split),
           ("hla_b_broad", b_broad),
           ("hla_c_split", c_split),
           ("hla_c_broad", c_broad),
           ("hla_drb1_split", drb1_split),
           ("hla_drb1_broad", drb1_broad)]
    
    df_hla_filtered = df_unacc_antigens_filtered.copy()
    
    for col, value in filters:
        if not pd.isnull(value):
            df_hla_filtered = df_hla_filtered.loc[(df_hla_filtered[col] == value) | (pd.isnull(df_hla_filtered[col]))].copy()

    if len(df_hla_filtered["normalized_freq"]) == 0:
            print("HLA allele filtering: The following combination is not represented in the HLA haplotype frequencies DataFrame:")
            print(f"a split:{a_split}, a broad:{a_broad}, b split:{b_split}, b broad:{b_broad}, c split:{c_split}, c broad:{c_broad}, drb1 split:{drb1_split}, drb1 broad:{drb1_broad} unnacceptable antigens:{unacceptable_antigens}")

            # some combinations are not in the haplotype frequency data
            # in these cases a completly new haplotype is generated
            # for this the unfiltered dataframe is used
            df_hla_filtered = df_unacc_antigens_filtered

    # the frequencies have to be normalized after filtering and the index has to be reset
    if any(not pd.isnull(val) for val in [a_broad, b_broad, c_broad, drb1_broad, drb1_split, a_split, b_split, c_split, unacceptable_antigens]):
        df_hla_filtered["normalized_freq"] = df_hla_filtered["normalized_freq"]/df_hla_filtered["normalized_freq"].sum()

        df_hla_filtered = df_hla_filtered.reset_index(drop=True)

    haplotype = df_hla_filtered.iloc[[random_numb_generator.choice(df_hla_filtered.index, p = df_hla_filtered["normalized_freq"])]]
    haplotype = haplotype[["hla_a_split", "hla_a_broad", "hla_b_split", "hla_b_broad", "hla_c_split", "hla_c_broad", "hla_drb1_split", "hla_drb1_broad"]]

    return list(haplotype.values[0])

def impute_hla_hapl(df, random_numb_generator):
    """
    Imputes missing HLA antigen values in specific columns by generating random haplotypes.

    Args:
    - df (pd.DataFrame): The DataFrame containing columns with missing HLA antigen values.
    - random_numb_generator (module): The random number generator to use.

    Returns:
    - pd.DataFrame: The updated DataFrame with missing HLA antigen values imputed in specified columns.

    Notes:
    - The function applies `gen_hla_haplotype` to impute missing values in two sets of HLA columns (`hla_a_split_1`, `hla_a_broad_1`, etc., and `hla_a_split_2`, `hla_a_broad_2`, etc.).
    - Filtering is based on available antigen information in each row, and random HLA haplotypes are generated only for rows with missing values in each respective set of columns.
    """

    df_out = df.copy(deep=True)

    necessary_hla_columns_1 = ["hla_a_broad_1", "hla_b_broad_1", "hla_c_broad_1", "hla_drb1_split_1"]
    hla_columns_1 = ["hla_a_split_1", "hla_a_broad_1", "hla_b_split_1", "hla_b_broad_1", "hla_c_split_1", "hla_c_broad_1", "hla_drb1_split_1", "hla_drb1_broad_1"]
    imputed1 = df.apply(lambda x: gen_hla_haplotype(df_hla_haplo_freq, 
                                                a_split=x["hla_a_split_1"], 
                                                a_broad=x["hla_a_broad_1"], 
                                                b_split=x["hla_b_split_1"],
                                                b_broad=x["hla_b_broad_1"],
                                                c_split=x["hla_c_split_1"],
                                                c_broad=x["hla_c_broad_1"],
                                                drb1_split=x["hla_drb1_split_1"],
                                                drb1_broad=x["hla_drb1_broad_1"],
                                                unacceptable_antigens=x["latest_unacceptable_antigens"],
                                                random_numb_generator = random_numb_generator)
                    if any(pd.isnull(x[col]) for col in necessary_hla_columns_1)
                    else [x[col] for col in hla_columns_1],
                    axis=1, result_type="expand")
    
    necessary_hla_columns_2 = ["hla_a_broad_2", "hla_b_broad_2", "hla_c_broad_2", "hla_drb1_split_2"]
    hla_columns_2 = ["hla_a_split_2", "hla_a_broad_2", "hla_b_split_2", "hla_b_broad_2", "hla_c_split_2", "hla_c_broad_2", "hla_drb1_split_2", "hla_drb1_broad_2"]
    imputed2 = df.apply(lambda x: gen_hla_haplotype(df_hla_haplo_freq, 
                                                a_split=x["hla_a_split_2"], 
                                                a_broad=x["hla_a_broad_2"], 
                                                b_split=x["hla_b_split_2"],
                                                b_broad=x["hla_b_broad_2"],
                                                c_split=x["hla_c_split_2"],
                                                c_broad=x["hla_c_broad_2"],
                                                drb1_split=x["hla_drb1_split_2"],
                                                drb1_broad=x["hla_drb1_broad_2"],
                                                unacceptable_antigens=x["latest_unacceptable_antigens"],
                                                random_numb_generator = random_numb_generator)
                    if any(pd.isnull(x[col]) for col in necessary_hla_columns_2)
                    else [x[col] for col in hla_columns_2],
                    axis=1, result_type="expand")

    df_out.loc[:, hla_columns_1] = pd.DataFrame(imputed1).values
    df_out.loc[:, hla_columns_2] = pd.DataFrame(imputed2).values

    return df_out

In [ ]:
hla_random_rng = np.random.default_rng(1234)
df_active_waiting_2006 = impute_hla_hapl(df_active_waiting_2006, hla_random_rng)
clear_output()

Testing for cases where the HLA type is in the unacceptible antigens list:

In [ ]:
df_active_waiting_2006.loc[df_active_waiting_2006[["hla_a_split_1",
                                                   "hla_a_broad_1",
                                                   "hla_b_split_1",
                                                   "hla_b_broad_1",
                                                   "hla_c_split_1",
                                                   "hla_c_broad_1",
                                                   "hla_drb1_split_1",
                                                   "hla_drb1_broad_1",
                                                   "hla_a_split_2",
                                                   "hla_a_broad_2",
                                                   "hla_b_split_2",
                                                   "hla_b_broad_2",
                                                   "hla_c_split_2",
                                                   "hla_c_broad_2",
                                                   "hla_drb1_split_2",
                                                   "hla_drb1_broad_2"]].isin(df_active_waiting_2006["latest_unacceptable_antigens"]).any(axis=1)]

## Load peak vPRAs and corresponding unacceptable antigens

In [ ]:
df_peak_vpra = pd.read_csv(data_path + "peak_vPRAs_freq.csv", sep = ";", decimal = ",", index_col = 0)

df_peak_vpra["unacceptable_antigens"] = df_peak_vpra["unacceptable_antigens"].apply(lambda x: x.split() if not pd.isnull(x) else x)

print(df_peak_vpra)

## Load Blood group frequencies

In [ ]:
donor_blood_grp_freq_dict = pd.read_csv(data_path + "donor_blood_grp.csv", sep = ";", decimal = ",", index_col = 0)

donor_blood_grp_freq_dict = donor_blood_grp_freq_dict.set_index("blood_grp_donor")["frequency"].to_dict()

print(donor_blood_grp_freq_dict)

In [ ]:
recipient_blood_grp_freq_dict = pd.read_csv(data_path + "recipient_blood_grp.csv", sep = ";", decimal = ",", index_col = 0)

recipient_blood_grp_freq_dict = recipient_blood_grp_freq_dict.set_index("blood_grp_rec")["frequency"].to_dict()

print(recipient_blood_grp_freq_dict)

## Load dialysis to registration time

In [ ]:
df_dial_to_reg = pd.read_csv(data_path + "time_dial_to_reg.csv", sep = ";", decimal = ",", index_col=0)

df_dial_to_reg["age_at_reg_bins"] = pd.cut(df_dial_to_reg["age_at_reg_waiting_list"], bins=range(0,101, 10), labels=False)*10

df_dial_to_reg.sort_values(by="age_at_reg_bins", inplace=True)

print(df_dial_to_reg)

In [ ]:
def make_kde_dict_dial_to_reg(df_time_dial_to_reg):
    dict_kde_dial_to_reg = {}
    for age_bin in df_time_dial_to_reg.age_at_reg_bins:
        kde = gaussian_kde(df_time_dial_to_reg.loc[df_time_dial_to_reg["age_at_reg_bins"] == age_bin, "time_dial_to_registration"])
        dict_kde_dial_to_reg.setdefault(age_bin, kde)
    return  dict_kde_dial_to_reg

dict_kde_dial_to_reg = make_kde_dict_dial_to_reg(df_dial_to_reg)

In [ ]:
def plot_kde_dial_to_reg(kde_dict, age_bin, ax):
    x = np.linspace(-100, 5000, 1000)
    y = kde_dict[age_bin](x)
    ax.plot(x, y)
    ax.set_ylim([0, 0.0025])
    ax.fill_between(x, y, alpha=0.5)
    ax.set_xlabel('Days')
    ax.set_ylabel('Density')
    ax.set_title(f'{age_bin} to {age_bin+10} years')

num_plots = len(dict_kde_dial_to_reg.keys())
num_cols = 3
num_rows = (num_plots + num_cols - 1) // num_cols

# Create subplots
fig, axs = plt.subplots(num_rows, num_cols, figsize=(8,8))
axs = axs.flatten()

# Plot each KDE in a separate subplot
for idx, age_bin in enumerate(dict_kde_dial_to_reg.keys()):
    plot_kde_dial_to_reg(dict_kde_dial_to_reg, age_bin, axs[idx])

for idx in range(num_plots, num_rows * num_cols):
    fig.delaxes(axs[idx])

plt.suptitle("KDE for time from dialysis to registration by age at list registration")
plt.tight_layout()
plt.show()

## Load DSO region frequencies for donors and recipients

In [ ]:
donor_region_freq = pd.read_csv(data_path + "donor_dso_reg.csv", sep = ";", decimal = ",", index_col = 0)
donor_region_freq = donor_region_freq.set_index("dso_region_donor")["frequency"].to_dict()
print(donor_region_freq)

In [ ]:
recipient_region_freq = pd.read_csv(data_path + "recipient_dso_reg.csv", sep = ";", decimal = ",", index_col = 0)
recipient_region_freq = recipient_region_freq.set_index("dso_region_rec")["frequency"].to_dict()
print(recipient_region_freq)

## Load newly registred patients per year

In [ ]:
new_pts_list = pd.read_csv(data_path + "new_reg_wait_per_year.csv", sep = ";", decimal = ",", index_col = 0)
print(new_pts_list)
new_pts_list = new_pts_list["n"].tolist()

## Load transplantations per year

Only post-mortem transplantations are counted.

In [ ]:
tx_per_year_list = pd.read_csv(data_path + "tx_per_year.csv", sep = ";", decimal = ",", index_col = 0)
print(tx_per_year_list)
tx_per_year_list = tx_per_year_list["n"].tolist()

## Load frequency of living donor donation

In [ ]:
living_donor_prob = pd.read_csv(data_path + "freq_living_donor.csv", sep = ";", decimal = ",", index_col = 0)

living_donor_prob["reg_waiting_list_year"] = living_donor_prob["reg_waiting_list_year"].str.replace("(", "").str.replace("]", "")

print(living_donor_prob)

In [ ]:
living_donor_prob = pd.read_csv(data_path + "freq_living_donor.csv", sep = ";", decimal = ",", index_col = 0)

# Convert the reg_waiting_list_year column to intervals
living_donor_prob["reg_waiting_list_year"] = living_donor_prob["reg_waiting_list_year"].str.replace("(", "").str.replace("]", "")
living_donor_prob[["start", "end"]] = living_donor_prob["reg_waiting_list_year"].str.split(",", expand=True).astype(str)

# Convert to float (handle 'Inf')
living_donor_prob["start"] = living_donor_prob["start"].replace("0", "0").astype(float)
living_donor_prob["end"] = living_donor_prob["end"].replace("Inf", "inf").astype(float)

# Now build a lookup dictionary:
living_donor_prob_lookup = {}

for age, row in living_donor_prob.iterrows():
    start, end = row["start"], row["end"]
    prob = row["living_donor_prob"]
    living_donor_prob_lookup[(age, start, end)] = prob

living_donor_prob = living_donor_prob_lookup

### Load age list for donors and recipients

In [ ]:
# loading the csv then taking the second column and coverting it to a list
list_pts_age = pd.read_csv(data_path + "waiting_list_reg_age.csv", sep = ";", decimal = ",").iloc[:,1].tolist()

list_donor_age = pd.read_csv(data_path + "donor_age.csv", sep = ";", decimal = ",").iloc[:,1].tolist()